# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [2]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

In [15]:
from pyspark.sql.functions import udf, desc,asc, sum as Fsum, col
from pyspark.sql.types import StringType, IntegerType

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [5]:
spark = SparkSession \
    .builder \
    .appName("Data Wrangling Quiz") \
    .getOrCreate()

In [6]:
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [8]:
user_log.show(5)

+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession| lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       Showaddywaddy|Logged In|  Kenneth|     M|          112| Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|          Lily Allen|Logged In|Elizabeth|     F|            7|    Chase|195.23873| free|Shreveport-Bossie...|   PUT

In [9]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [10]:
user_log.count()

10000

In [12]:
user_log.select("page").dropDuplicates().sort("page").show()

+----------------+
|            page|
+----------------+
|           About|
|       Downgrade|
|           Error|
|            Help|
|            Home|
|           Login|
|          Logout|
|        NextSong|
|   Save Settings|
|        Settings|
|Submit Downgrade|
|  Submit Upgrade|
|         Upgrade|
+----------------+



In [16]:
user_log.where(col("userId") == "").select("page").dropDuplicates().sort("page").show()

+-----+
| page|
+-----+
|About|
| Help|
| Home|
|Login|
+-----+



In [17]:
path = "data/sparkify_log_small.json"
df = spark.read.json(path)

# Question 1

Which page did user id "" (empty string) NOT visit?

In [18]:
# TODO: write your code to answer question 1
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [24]:
all_pages = df.select('page').dropDuplicates()
blank_pages = df.filter(df.userId == '')\
                .select('page')\
                .dropDuplicates()
print(all_pages.collect())
print(blank_pages.collect())

[Row(page='Submit Downgrade'), Row(page='Home'), Row(page='Downgrade'), Row(page='Logout'), Row(page='Save Settings'), Row(page='About'), Row(page='Settings'), Row(page='Login'), Row(page='NextSong'), Row(page='Help'), Row(page='Upgrade'), Row(page='Error'), Row(page='Submit Upgrade')]
[Row(page='Home'), Row(page='About'), Row(page='Login'), Row(page='Help')]


In [25]:
for row in set(all_pages.collect()) - set(blank_pages.collect()):
    print(row.page)

Logout
Upgrade
Downgrade
Settings
Submit Downgrade
Error
Submit Upgrade
NextSong
Save Settings


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [2]:
# TODO: use this space to explore the behavior of the user with an empty string


Perhaps it represents users who have not signed up yet or who are signed out and are about to log in.

# Question 3

How many female users do we have in the data set?

In [28]:
# TODO: write your code to answer question 3
df.filter(col('gender') == 'F')\
    .select('userId')\
    .dropDuplicates()\
    .count()

462

# Question 4

How many songs were played from the most played artist?

In [29]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [32]:
# TODO: write your code to answer question 4
df.filter(df.page == 'NextSong') \
    .groupBy('Artist')\
    .agg({'Artist' :'count'})\
    .withColumnRenamed('count(Artist)', 'Artistcount')\
    .sort(desc('Artistcount'))\
    .show(5)

+--------------------+-----------+
|              Artist|Artistcount|
+--------------------+-----------+
|            Coldplay|         83|
|       Kings Of Leon|         69|
|Florence + The Ma...|         52|
|            BjÃÂ¶rk|         46|
|       Dwight Yoakam|         45|
+--------------------+-----------+
only showing top 5 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [34]:
df.select('artist', 'page').show(3)

+--------------------+--------+
|              artist|    page|
+--------------------+--------+
|       Showaddywaddy|NextSong|
|          Lily Allen|NextSong|
|Cobra Starship Fe...|NextSong|
+--------------------+--------+
only showing top 3 rows



In [ ]:
# TODO: write your code to answer question 5
function = udf(lambda ishome: int(ishome == 'Home'),IntegerType())

user_window = Window.partitionBy('userId')\
                    .orderBy(desc('ts'))\
                    .rangeBetween(Window.unboundedPreceding, 0)
cusum = df.filter((df.page == 'NextSong') | (df.page == 'Home')) \
        .select('userId', 'ts', 'page')
        .withColumn('homevisit', function(col('page')))